## Investigación Profunda

¡Uno de los casos de uso clásicos de Agentic en diferentes empresas! ¡Esto es enorme!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Implicaciones comerciales</h2>
            <span style="color:#00bfff;">Un agente de Investigación Profunda es ampliamente aplicable a cualquier área de negocio y a tus actividades diarias. ¡Puedes usarlo tú mismo!
            </span>
        </td>
    </tr>
</table>

In [20]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## Herramientas alojadas de OpenAI

El SDK de agentes de OpenAI incluye las siguientes herramientas alojadas:

- La herramienta `WebSearchTool` permite a un agente buscar en la web.
- La herramienta `FileSearchTool` permite recuperar información de sus almacenes de vectores de OpenAI.
- La herramienta `ComputerTool` permite automatizar tareas informáticas como tomar capturas de pantalla y hacer clic.

### Nota importante: Costo de la API de la herramienta `WebSearchTool`

Esto me cuesta 2,5 centavos por llamada para la herramienta `WebSearchTool` de OpenAI. Esto puede suponer entre 2 y 3 dólares para los próximos dos laboratorios. Usaremos herramientas de búsqueda de bajo coste con otras plataformas, así que no dude en omitir esta opción si le preocupa el coste.

Los costes están disponibles aquí: https://platform.openai.com/docs/pricing#web-search

In [3]:
INSTRUCTIONS = "Eres un asistente de investigación. Dado un término de búsqueda, buscas en la web ese término y \
producí una descripción concisa de los resultados. La descripción debe tener 2-3 párrafos y menos de 300 \
palabras. Captura los puntos principales. Escribe de manera concisa, no es necesario tener frases completas o buena \
gramática. Esto será consumido por alguien que está sintetizando un informe, por lo que es vital que captures el \
esencia y ignores cualquier fluff. No incluyas ningún comentario adicional más que la descripción en sí."

search_agent = Agent(
    name="Agente de búsqueda",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Últimos frameworks de agentes de IA en 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

En 2025, emergen varios frameworks de agentes de inteligencia artificial (IA) que facilitan la creación y gestión de sistemas autónomos. LangGraph, desarrollado por los creadores de LangChain, ofrece un marco modular para construir agentes basados en modelos de lenguaje (LLM), permitiendo la ejecución de flujos de trabajo complejos con encadenamiento de tareas y gestión de memoria. CrewAI, respaldado por Andrew Ng, permite la creación de equipos de IA donde cada agente tiene funciones y objetivos específicos, colaborando para resolver tareas complejas. AutoGen de Microsoft es un framework de código abierto que facilita la cooperación entre múltiples agentes de IA para resolver tareas, basado en el modelo de actores donde cada agente es un actor independiente. FlowiseAI es una herramienta de código bajo que permite a los desarrolladores crear flujos de orquestación LLM y agentes de IA personalizados mediante una interfaz visual, sin necesidad de escribir grandes cantidades de código. OpenAI Swarm es un marco experimental y liviano diseñado para simplificar la creación de flujos de trabajo de múltiples agentes, utilizando estructuras JSON para la integración y ejecución de agentes. ([kwfoundation.org](https://kwfoundation.org/blog/2025/02/19/el-agente-de-inteligencia-artificial-significara-el-fin-del-saas/?utm_source=openai))

Además, surgen frameworks como Eliza, un sistema operativo de agentes de IA compatible con Web3, que facilita la integración de aplicaciones Web3 en funcionalidades de agentes de IA. AutoAgent es un framework totalmente automatizado y sin código que permite a los usuarios crear y desplegar agentes de LLM mediante lenguaje natural, operando como un sistema operativo de agentes autónomos. Autono propone un framework de agentes autónomos altamente robusto basado en el paradigma ReAct, diseñado para resolver tareas complejas mediante toma de decisiones adaptativa y colaboración entre múltiples agentes. Manus, desarrollado por la startup china Monica, es un agente de IA autónomo capaz de realizar tareas complejas en línea sin intervención humana directa, incluyendo redacción e implementación de código. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai), [arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai), [arxiv.org](https://arxiv.org/abs/2504.04650?utm_source=openai), [es.wikipedia.org](https://es.wikipedia.org/wiki/Manus_%28agente_de_IA%29?utm_source=openai))

Estos frameworks representan avances significativos en la creación de agentes de IA autónomos y colaborativos, ofreciendo herramientas y estructuras que facilitan el desarrollo de sistemas inteligentes capaces de ejecutar tareas complejas de forma independiente. 

### Como siempre, revisamos la traza:

https://platform.openai.com/traces

### Ahora utilizaremos salidas estructuradas e incluiremos una descripción de los campos

In [14]:
# Consulte la nota anterior sobre el costo de WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"Eres un asistente de investigación útil. Dado un término de búsqueda, \
produce un conjunto de búsquedas web para realizar para responder la consulta. \
Salida: {HOW_MANY_SEARCHES} términos para consultar."

# Usamos Pydantic para definir el esquema de nuestra respuesta; esto se conoce como "Salidas Estructuradas".
# ¡Muchas gracias al estudiante Wes C. por descubrir y corregir un error desagradable!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Tu razonamiento de por qué esta búsqueda es importante para la consulta.")

    query: str = Field(description="El término de búsqueda para usar para la búsqueda web.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="Una lista de búsquedas web a realizar para responder la consulta.")


planner_agent = Agent(
    name="Agente de planificación",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [15]:
message = "Últimos frameworks de agentes de IA en 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='Para identificar los frameworks de agentes de IA más recientes y populares en 2025, y entender sus características y utilidades.', query='últimos frameworks de agentes de IA 2025'), WebSearchItem(reason='Verificar las tendencias emergentes en la tecnología de IA y qué frameworks están siendo adoptados en la industria.', query='tendencias frameworks agentes de IA 2025'), WebSearchItem(reason='Recopilar opiniones y análisis de expertos sobre los frameworks de IA más eficaces y sus aplicaciones en 2025.', query='análisis expertos frameworks agentes de IA 2025'), WebSearchItem(reason='Discover new and innovative AI agent frameworks launched or gaining popularity in 2025.', query='nuevos frameworks de agentes de IA 2025'), WebSearchItem(reason='Comparar diferentes frameworks de IA en términos de funcionalidad y rendimiento para el año 2025.', query='comparación frameworks de agentes de IA 2025'), WebSearchItem(reason='Examinar los informes de mercado sobre he

In [7]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Envía un correo electrónico con el asunto y el cuerpo HTML proporcionados """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("juangabriel@frogames.es") # Cambiar a tu correo electrónico verificado
    to_email = To("juangabriel@frogames.es") # Cambiar a tu correo electrónico
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
send_email

FunctionTool(name='send_email', description='Envía un correo electrónico con el asunto y el cuerpo HTML proporcionados', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x108b73060>, strict_json_schema=True)

In [16]:
INSTRUCTIONS = """Eres capaz de enviar un correo electrónico HTML bien formateado basado en un informe detallado.
Se te proporcionará un informe detallado. Debes usar tu herramienta para enviar un correo electrónico, proporcionando el 
informe convertido en HTML limpio, bien presentado con un asunto adecuado."""

email_agent = Agent(
    name="Agente de correo electrónico",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [17]:
INSTRUCTIONS = (
    "Eres un investigador senior encargado de escribir un informe coherente para una consulta de investigación. "
    "Se te proporcionará la consulta original y algunas investigaciones iniciales realizadas por un asistente de investigación.\n"
    "Primero, debes elaborar un esquema para el informe que describa la estructura y "
    "flujo del informe. Luego, genera el informe y devuelve ese como tu salida final.\n"
    "La salida final debe estar en formato markdown, y debe ser larga y detallada "
    "para 5-10 páginas de contenido, al menos 1000 palabras."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="Un resumen de 2-3 párrafos de los resultados.")

    markdown_report: str = Field(description="El informe final")

    follow_up_questions: list[str] = Field(description="Temas sugeridos para investigar más")


writer_agent = Agent(
    name="Agente de escritura",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### Las siguientes 3 funciones planificarán y ejecutarán la búsqueda, utilizando planner_agent y search_agent

In [11]:
async def plan_searches(query: str):
    """ Utilice planner_agent para planificar qué búsquedas ejecutar para la consulta """
    print("Planificando búsquedas...")
    result = await Runner.run(planner_agent, f"Consulta: {query}")
    print(f"Se realizarán {len(result.final_output.searches)} búsquedas")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Llama a search() para cada elemento en el plan de búsqueda """
    print("Buscando...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Búsqueda finalizada")
    return results

async def search(item: WebSearchItem):
    """ Usa el agente de búsqueda para ejecutar una búsqueda web para cada elemento en el plan de búsqueda """
    input = f"Término de búsqueda: {item.query}\nRazón para buscar: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### Las siguientes 2 funciones escriben un informe y lo envían por correo electrónico.

In [19]:
async def write_report(query: str, search_results: list[str]):
    """ Usa el agente de escritura para escribir un informe basado en los resultados de la búsqueda """
    print("Pensando sobre el informe...")
    input = f"Consulta original: {query}\nResultados de búsqueda resumidos: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Informe finalizado")
    return result.final_output

async def send_email(report: ReportData):
    """ Usa el agente de correo electrónico para enviar un correo electrónico con el informe """
    print("Escribiendo correo electrónico...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Correo electrónico enviado")
    return report

### ¡Hora del espectáculo!

In [18]:
query = "Últimos frameworks de agentes de IA en 2025"

with trace("Investigación"):
    print("Iniciando investigación...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("¡Felicidades!")




Iniciando investigación...
Planificando búsquedas...
Se realizarán 20 búsquedas
Buscando...
Búsqueda finalizada
Pensando sobre el informe...
Informe finalizado
Escribiendo correo electrónico...


AttributeError: 'function' object has no attribute 'name'

### Como siempre, revisa la traza del resultado:

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Felicitaciones por tu progreso y una solicitud</h2>
<span style="color:#00cc00;">Has llegado a un momento importante con el curso; has creado un agente valioso usando uno de los frameworks de agentes más recientes. Has mejorado tus habilidades y has descubierto nuevas posibilidades comerciales. ¡Tómate un momento para celebrar tu éxito!<br/><br/>Si puedes calificar el curso en esta plataforma, te lo agradecería muchísimo: es la forma más importante en que se decide si lo muestra a otros y marca una gran diferencia.<br/><br/>Y otro recordatorio para que <a href="https://www.linkedin.com/in/juan-gabriel-gomila-salas/">conectes conmigo en LinkedIn</a> si lo deseas. Si quieres publicar sobre tu progreso en el curso, etiquétame y participaré para aumentar tu visibilidad. </span>
        </td>
    </tr>